# Time Series Analysis Mini Project:  Let's explore my steps, and if we are feeling froggy maybe use FB prophet to model...

In [24]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

# Acquire!

### Imported steps from my personal Iphone pedometer. 

#### Steps to repeat:
1. Export from Iphone pedometer in settings
2. Save to file
3. Upload using pandas!

In [6]:
df = pd.read_csv("Export.csv")

In [9]:
df.head(2)

,Date,Steps,Distance,Floors Ascended
0,8/16/20,14656,6.7,0
1,8/15/20,10103,4.7,8


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589 entries, 0 to 588
Data columns (total 4 columns):
Date               589 non-null object
Steps              589 non-null int64
Distance           589 non-null float64
Floors Ascended    589 non-null int64
dtypes: float64(1), int64(2), object(1)
memory usage: 18.5+ KB


# Prepare!!

### Initial thoughts:
- This is a really simple data frame
- Only 3 columns to explore in additon to datetime

### Action Items:
- Lets rename some columns in a pythonic manner (h/t Ryan Orsinger)
- Lets set the index as datetime so we can do some time series analysis.

### Personal Notes on Time-Series Analysis
- For the past couple of projects I have been blowing off TSA by moving the target variable from the next time-series row into the current row.  For this mini-project I wanted to do "pure" TSA.
- I also have been anxious to explore FB prophet, so I will be importing and exploring here.


In [15]:
#rename columns
df = df.rename(columns={'Date':'date',
                  'Steps': 'steps',
                    'Distance': 'distance',
                  'Floors Ascended': 'floors_ascended'})

In [27]:
# # set index to datetime
df['date'] = pd.to_datetime(df.date)
df = df.sort_values('date').set_index('date')

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 589 entries, 2018-08-26 to 2020-08-16
Data columns (total 3 columns):
steps              589 non-null int64
distance           589 non-null float64
floors_ascended    589 non-null int64
dtypes: float64(1), int64(2)
memory usage: 18.4 KB
